# Introduction

In this notebook I'll apply different EDA (Exploratory Data Analysis) techniques on the [Graduate Admission 2 data](https://www.kaggle.com/mohansacharya/graduate-admissions).

The goal in this data is to predict the *student's chance of admission* to a postgraduate education, given several *predictor* variables for the student.

# Import libraries

In [ ]:
import pandas as pd
import plotly.express as px
from scipy import stats

Defing some color values:

In [ ]:
red_color_hex = "#E01427"
green_color_hex = "#158A24"
blue_color_hex = "#04668C"

# Load data

There are two data files:
- `Admission_Predict.csv`
- `Admission_Predict_Ver1.1.csv`
Will use the second one, since it contains more data points.

In [ ]:
df = pd.read_csv("data/Admission_Predict_Ver1.1.csv")

According to the dataset author on Kaggle, the columns in this data represents:
- `GRE Score`: The Graduate Record Examinations is a standardized test that is an admissions requirement for many graduate schools in the United States and Canada.
- `TOEFL Score`: Score in TOEFL exam.
- `University Rating`: Student undergraduate university ranking.
- `SOP`: Statement of Purpose strength.
- `LOR`: Letter of Recommendation strength.
- `CGPA`: Undergraduate GPA.
- `Research`: Whether student has research experience or not.
- `Chance of Admit`: Admission chance.

# Getting to know the data

In this section, we'll take a quick look at the data, to see how many row are there, and whther there are any missing values or not, to decie what kind of preprocessing will be needed.

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

The dataset consists of 500 samples and 9 columns: 8 *predictors* and one *target* variable.

There are no missing values (which is a very good thing!), but some column names need to be cleaned, and the `Serial No.` must be removed, as it has nothing to do with the student's overall admission chance.

Lookin at the `dtypes` it seems that all columns are in the correct data type, discrete columns are in `int64` and continuous in `float64`.

# Data cleaning and Preprocessing

As stated in the previous section, only few *cleaning* will be performed, mainly:
- remove extra whitespace from column names.
- drop `Serial No.` column
- convert `Research` column to bool.

In [ ]:
df.columns

Pandas has a great feature which allows us to apply multiple functions on the `DataFrame` in a sequential order: the [pipe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pipe.html) method.

Here, I'll define two separate functions for applying each processing step, and then call them using the `pipe` function.

In [ ]:
def read_data():
    temp_df = pd.read_csv("data/Admission_Predict_Ver1.1.csv")
    return temp_df

In [ ]:
def normalize_column_names(temp_df):
    return temp_df.rename(
        columns={"LOR ": "LOR", "Chance of Admit ": "Chance of Admit"}
    )

In [ ]:
def drop_noisy_columns(temp_df):
    return temp_df.drop(columns=["Serial No."])

In [ ]:
def normalize_dtypes(temp_df):
    return temp_df.astype({"Research": bool, "University Rating": str})

In [ ]:
def sort_uni_ranking(temp_df):
    return temp_df.sort_values(by="University Rating")

Now, we plug them together:

In [ ]:
df = (
    read_data()
    .pipe(normalize_column_names)
    .pipe(drop_noisy_columns)
    .pipe(normalize_dtypes)
    .pipe(sort_uni_ranking)
)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.dtypes

We *cleaned* the data with a *clean* code!

# Exploratory Data Analysis (EDA)

In this section, we'll explore the data *visually* and summarize it using *descriptive statistic* methods.

To keep things simpler, we'll divide this section into three subsections:
1. Univariate analysis: in this section we'll focus only at one variable at a time, and study the variable descriptive statistics with some charts like: Bar chart, Line chart, Histogram, Boxplot, etc ..., and how the variable is distributed, and if there is any *skewness* in the distribution.
2. Bivariate analysis: in this section we'll study the relation between *two* variables, and present different statistics such as Correlation, Covariance, and will use some other charts like: scatterplot, and will make use of the `hue` parameter of the previous charts.
3. Multivariate analysis: in this section we'll study the relation between three or more variables, and will use additional type of charts, such as parplot.

## Univariate Analysis

Here in this section, will perform analysis on each variable individually, but according to the variable type different methods and visualization will be used, main types of variables:
- Numerical: numerical variables are variables which measures things like: counts, grades, etc ..., and they don't have a *finite* set of values, and they can be divided to:
    - Continuous: continuous variables are continous measurements such as weight, height.
    - Discrete: discrete variables represent counts such as number of children in a family, number of rooms in a house.
- Categorical: a categorical variable is a variable which takes one of a limited values, and it can be further divided to:
    - Nominal: nominal variable has a finite set of possible values, which don't have any ordereing relation among them, like countries, for example we can't say that `France` is higher than `Germany`: `France` > `Germany`, therfore, there's no sense of ordering between the values in a noinal variable.
    - Ordinal: in contrast to `Nominal` variable, ordinal varible defines an ordering relation between the values, such as the student performance in an exam, which can be: `Bad`, `Good`, `Very Good`, and `Excellent` (there's an ordering relation among theses values, and we can say that `Bad` is lower than `Good`: `Bad` < `Good`)
    - Binary: binary variables are a special case of nominal variables, but they only have *two* possible values, like admission status which can either be `Accepted` or `Not Accepted`.

resources:
- [Variable types and examples](https://www.statsandr.com/blog/variable-types-and-examples/)
- [What is the difference between ordinal, interval and ratio variables? Why should I care?](https://www.graphpad.com/support/faq/what-is-the-difference-between-ordinal-interval-and-ratio-variables-why-should-i-care/)

Let's see what are the types of variables in our dataset:

In [ ]:
df.describe()

- Discrete: `GRE Score` and `TOEFL Score` are discrete variables.
- Continuous: `CGPA` and `Chance of Admit` are continuous variables.
- Ordinal: `University Rating`, `SOP` and `LOR` are ordinal variables.
- Binary: `Research` is a binary variable.

### `GRE Score`

The `GRE Score` is a discrete variable.

In [ ]:
df["GRE Score"].describe()

In [ ]:
print(df["GRE Score"].mode())

In [ ]:
print(stats.skew(df["GRE Score"]))

In [ ]:
fig = px.histogram(
    df,
    x="GRE Score",
    color_discrete_sequence=[blue_color_hex],
    nbins=20,
    marginal="box",
    title="Distribution of the <b>GRE Score</b> variable",
)

fig.show()

We can conclude from the previous chart the following:

- The GRE scores are *very close* to a normal distribution, with a small negative skewnewss (left skewed).
- The most common scores are between `310` and `325`.
- The average score is `316` with a standard deviation of `11.2`.
- There are no outliers.

This variable doesn't need any further processing.

### `TOEFL Score`

The `TOEFL Score` is a discrete variable.

In [ ]:
df["TOEFL Score"].describe()

In [ ]:
print(df["TOEFL Score"].mode())

In [ ]:
print(stats.skew(df["TOEFL Score"]))

In [ ]:
fig = px.histogram(
    df,
    x="TOEFL Score",
    color_discrete_sequence=[blue_color_hex],
    marginal="box",
    nbins=15,
    title="Distribution of <b>TOEFL Score</b> variable",
)

fig.show()

In [ ]:
df["TOEFL Score"].value_counts()[:4]

From the previous chart, we can conclude:
- TOEFL scores are also normally distributed, with a small positive (right skewness).
- The average TOEFL score is `107` with a standard deviation `6`.
- The most common scores are: `110`, `105`, `104` and `112`.
- There are no outliers.

The variable doesn't need any further processing.

### `University Rating`

The `University Rating` is an ordinal variable, it represents the student's undergraduate university ranking on a scale 1-5.

In [ ]:
df["University Rating"].value_counts()

In [ ]:
temp_df = df.groupby(by="University Rating", as_index=False).agg(
    counts=pd.NamedAgg(column="University Rating", aggfunc="count")
)

In [ ]:
temp_df["University Rating"] = temp_df["University Rating"].astype(str)

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="University Rating",
    y="counts",
    color="University Rating",
    color_discrete_sequence=px.colors.qualitative.D3,
    title="Distribution of <b>University rating</b> variable",
)

fig.show()

We can see that the most common rating is in the middle: `3`. The chart shows that the ratings are distributed in a similar fashion to the normal distrbution.

### `SOP`

`SOP` stands for the strength of *Statement of Purpose* which is a necessary document for graduate applications. The values were (mostly) entered by the students, and it's on scale 1-5, so this is an ordinal variable.

In [ ]:
df["SOP"].value_counts()

In [ ]:
temp_df = df.groupby(by="SOP", as_index=False).agg(
    counts=pd.NamedAgg(column="SOP", aggfunc="count")
)

In [ ]:
temp_df["SOP"] = temp_df["SOP"].astype(str)

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="SOP",
    y="counts",
    color="SOP",
    color_discrete_sequence=px.colors.qualitative.Prism,
    title="Distribution of <b>SOP</b> variable",
)

fig.show()

Most students estimated the strength of their *Statement of Purpose* between `3` and `4`.

### `LOR`

`LOR` stands for the strength of *Letter of Recommendation*. The values were (mostly) entered by the students, and it's on scale 1-5, so this is an ordinal variable.

In [ ]:
df["LOR"].value_counts()

In [ ]:
temp_df = df.groupby(by="LOR", as_index=False).agg(
    counts=pd.NamedAgg(column="LOR", aggfunc="count")
)

In [ ]:
temp_df["LOR"] = temp_df["LOR"].astype(str)

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="LOR",
    y="counts",
    color="LOR",
    color_discrete_sequence=px.colors.qualitative.Prism,
    title="Distribution of <b>LOR</b> variable",
)

fig.show()

Most of the students rated the strength of ther *Letter of Recommendation* between `3` and `4`.

### `CGPA`

The `CGPA` stands for the student's *cumulative grade point average*, which represents the average of grade points obtained in all the subjects by the student.

It's a continuous variable, on a scale 0-10.

In [ ]:
df["CGPA"].describe()

In [ ]:
print(stats.skew(df["CGPA"]))

In [ ]:
fig = fig = px.histogram(
    data_frame=df,
    x="CGPA",
    marginal="box",
    color_discrete_sequence=[blue_color_hex],
    nbins=12,
    title="Distribution of <b>CGPA</b> variable",
)

fig.show()

As we can see, this variable is *very* close to a normal distribution, with a small negative (left) skewness, and there are no outliers.

### `Research`

The `Research` variable indicates whether the student has any research experience or not, so it's a `Binary` variable.

Although, it would be better to have a variable like `Research duration` which expresses for how long was the student involved in a research activity.

In [ ]:
df["Research"].value_counts()

In [ ]:
temp_df = df.groupby(by="Research", as_index=False).agg(
    counts=pd.NamedAgg(column="Research", aggfunc="count")
)

In [ ]:
fig = px.bar(
    data_frame=temp_df,
    x="Research",
    y="counts",
    color="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    color_continuous_scale=px.colors.qualitative.D3,
    title="Distribution of <b>Research</b> variable",
)

fig.show()

From this plot, we can see that the number of students who have a research experience is *almost* equal to the number of students who don't.

Later, we'll study the relation of this variable with other variables.

### `Chance of Admit`

Quoting the dataste author from this [thread](https://www.kaggle.com/mohansacharya/graduate-admissions/discussion/79063#464899):

> chance of admit is a parameter that was asked to individuals (some values manually entered) before the results of the application

So thie column is not an actual *probability of admission* estimated by the universities or something, rather, it's an estimation by the student themselves of how likely they'll be admitted to the university.

In [ ]:
df["Chance of Admit"].describe()

In [ ]:
print(stats.skew(df["Chance of Admit"]))

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="Chance of Admit",
    marginal="box",
    color_discrete_sequence=[blue_color_hex],
    title="Distribution of <b>Chance of Admit</b> variable",
)

fig.show()

In [ ]:
df[df["Chance of Admit"] < 0.36]

The plot shows that most student estimated their chance of admission between `0.7` and `0.75`.

The distribution is *moderately* skewed to the left with a negative skew value `-0.29`.

There are also two outlier values `0.34`.

## Bivariate Analysis

In this section, we'll focus on studying the relationship between two different variables, to answer different question, like
- What is the relation between variable `x` and variable `y`? is it linear or non-linear?
- In case of a linear relation, is positive linear relation or negative linear relation? and how *strong* is the relation?
- How the distribution for two variables changes?

### Correlation matrix

We'll start off by computing the correlation matrix using `.corr` method of pandas, which computes the pairwise correlation of columns.

The method used for calculating the correlation between two variables `x` and `y` is the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient).

The pearson coefficient is a measure of the linear correlation between two variables `x` and `y`, and it takes values between `-1` and `+1`.

A negative value indicates a negative correlation (i.e. when one variable *increases* the other *decreases*), and a positive value is the opposite (the two variables *increases*/*decreases* at the same time)

![Correlation coefficient](https://upload.wikimedia.org/wikipedia/commons/3/34/Correlation_coefficient.png)

Here, we'll compute the correlations only for `GRE Score`, `TOEFL Score` and `CGPA` variables, because they are *numeric* variables, and they weren't estimated by the students themselves.

In [ ]:
numeric_cols = ["GRE Score", "TOEFL Score", "CGPA"]

In [ ]:
corr = df[numeric_cols].corr()

In [ ]:
fig = px.imshow(
    corr,
    color_continuous_scale="PuBu",
    color_continuous_midpoint=0.6,
    title="Correlation matrix",
)

fig.show()

We can see from the correlation matrix that the three variables have a strong positive correlation. We'll look closer at the relations between variables using scatter plots.

### Scatter plot

Scatter plots are a good way to show the spread of points for two variables `x` and `y`, and view the relation between the two variables (e.g. linear, non-linear, ...) and the trend of the linear relation (positive, negative)

An easy way to show multiple scatter plots on the same figure is either using `scatter_matrix` or `pairplot`.

In [ ]:
fig = px.scatter_matrix(
    df,
    dimensions=numeric_cols,
    color_discrete_sequence=[blue_color_hex],
    title="Scatter matrix of student's <b>TOEFL Score</b>, <b>GRE Score</b>, and <b>CGPA</b>",
)

fig.update_traces(diagonal_visible=False)

fig.show()

It's evident from these plots that the relation between the variables is positive linear relation, with some outlier points, and they all have an upward trend.

Let's show the scatter for each two variables at a time:

#### `TOEFL Score` vs. `GRE Score`

In [ ]:
corr_value = df["TOEFL Score"].corr(df["GRE Score"])

fig = px.scatter(
    data_frame=df,
    x="TOEFL Score",
    y="GRE Score",
    marginal_x="histogram",
    marginal_y="histogram",
    trendline="ols",
    color_discrete_sequence=[blue_color_hex],
    trendline_color_override=red_color_hex,
    title=f"Correlation between <b>TOEFL Score</b> and <b>GRE Score</b> is: {corr_value:.2f}",
)

fig.show()

#### `TOEFL Score` vs. `CGPA`

In [ ]:
corr_value = df["TOEFL Score"].corr(df["CGPA"])

fig = px.scatter(
    data_frame=df,
    x="TOEFL Score",
    y="CGPA",
    marginal_x="histogram",
    marginal_y="histogram",
    trendline="ols",
    color_discrete_sequence=[blue_color_hex],
    trendline_color_override=red_color_hex,
    title=f"Correlation between <b>TOEFL Score</b> and <b>CGPA</b> is: {corr_value:.2f}",
)

fig.show()

#### `GRE Score` vs. `CGPA`

In [ ]:
corr_value = df["GRE Score"].corr(df["CGPA"])

fig = px.scatter(
    data_frame=df,
    x="GRE Score",
    y="CGPA",
    marginal_x="histogram",
    marginal_y="histogram",
    trendline="ols",
    color_discrete_sequence=[blue_color_hex],
    trendline_color_override=red_color_hex,
    title=f"Correlation between <b>GRE Score</b> and <b>CGPA</b> is: {corr_value:.2f}",
)

fig.show()

From the previous three charts we can say that: students who perform well in their `TOEFL` exams tend to also perform well in `GRE` exams, and they *mostly* have high `GPA` (higher than 9).

### Bivariate distributions

Another way to study the relation between two variables is with 2D Histograms (distribution).

Just like the distributions we used in the **Univariate Analysis** section, we can show the distribution for two variables `x` and `y`, which would give us better insights on how much the values from the two variables overlap, and show cluster regions in the 2D space.

Compared to scatter plots, 2D histograms are better at handling large amounts of data, as they use rectangular bins, and count the number of points withing each bin.

#### `TOEFL Score` vs. `GRE Score`

In [ ]:
fig = px.density_heatmap(
    data_frame=df,
    x="TOEFL Score",
    y="GRE Score",
    color_continuous_scale=px.colors.sequential.Blugrn,
    title="Joint distribution of <b>TOEFL Score</b> and <b>GRE Score</b> variables",
)

fig.show()

We can see from this chart some *clusters* (regions).

For example there are two clusters of students who scored between `110` and `115` in the `TOEFL` exam and between `320` and `330` in the `GRE` exam. These two clusters account for about 100 students (which is 20% of the total dataset).

#### `TOEFL Score` vs. `CGPA`

In [ ]:
fig = px.density_heatmap(
    data_frame=df,
    x="TOEFL Score",
    y="CGPA",
    color_continuous_scale=px.colors.sequential.Blugrn,
    title="Joint distribution of <b>TOEFL Score</b> and <b>CGPA</b> variables",
)

fig.show()

This chart shows that about `170` students has `TOEFL` score in range `[105-115]` and their `CGPA` is in range `[8-9]`

#### `GRE Score` vs. `CGPA`

In [ ]:
fig = px.density_heatmap(
    data_frame=df,
    x="GRE Score",
    y="CGPA",
    color_continuous_scale=px.colors.sequential.Blugrn,
    title="Bivariate distribution of <b>GRE Score</b> and <b>CGPA</b> variables",
)

fig.show()

This chart shows that almost `120` students has `GRE` scores in the range `[310-320]` and their `CGPA` is in thae range `[8-9]`.

### `Research`

One interesting question would be: How does research experience affects other variables? Do students who have research experience have beeter *CGPA*? Do they have better scores in *TOEFL* or *GRE* (or both)?

We can display the same distributions we used in the *Univariate analysis* section, with conditioning on `Research` variable:

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="TOEFL Score",
    color="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    barmode="group",
    title="Conditional distribution of <b>TOEFL Score</b> variable on the <b>Research</b> variable",
)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="GRE Score",
    color="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    barmode="group",
    title="Conditional distribution of <b>GRE Score</b> variable on the <b>Research</b> variable",
)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="CGPA",
    color="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    barmode="group",
    title="Conditional distribution of <b>CGPA</b> variable on the <b>Research</b> variable",
)

fig.show()

We can see that students who engage in research activities and have research experience, tend to perform better in both *TOEFL* and *GRE* exams, and they have higher *GPA*, in comparison to students who have no research experience.

### `University Rating`

The `University Rating` variable represents the ranking of the university from which the student graduated.

We might ask: How does the relation between student different scores changes for different university ranking?

Let's see how the university ranking affects student's scores and GPA:

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="TOEFL Score",
    color="University Rating",
    barmode="group",
    color_discrete_sequence=px.colors.sequential.Burg,
    title="Conditional distribution of <b>TOEFL Score</b> variable on the <b>University Rating</b> variable",
)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="GRE Score",
    color="University Rating",
    color_discrete_sequence=px.colors.sequential.Burg,
    barmode="group",
    title="Conditional distribution of <b>GRE Score</b> variable on the <b>University Rating</b> variable",
)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="CGPA",
    color="University Rating",
    color_discrete_sequence=px.colors.sequential.Burg,
    barmode="group",
    title="Conditional distribution of <b>CGPA</b> variable on the <b>University Rating</b> variable",
)

fig.show()

The university ranking plays an important role in student's scores and GPA, we can observe that students who go to higher ranking universities, have higher scores in the *TOEFL* and *GRE* exams, and they have higher *GPA*.

## Multivariate Analysis

So far, all the plots we used before were used either to explore one variable, or to show the relation between a pair of variables.

However, we are often interested in answering the question: **How does the relation between two variables changes as a function of a *third* variable?**

In this section, we'll focus on answering these kinds of questions, where we'll use similar plots to the ones we used before, with conditioning on other variable.

### Scatter matrix with `Research`

We start by plotting the scatter matrix for variables `TOEFL Score`, `GRE Score` and `CGPA` (just as we did in bivariate analysis section), with showing the relation to `Research` variable with color coding.

This will draw an overall picture of how these variables are related to each other, and how this relation changes when whether student has a research experience or not.

In [ ]:
fig = px.scatter_matrix(
    data_frame=df,
    dimensions=numeric_cols,
    color="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    title="Scatter matrix for <b>TOEFL Score</b>, <b>GRE Score</b>, and <b>CGPA</b> conditioning on <b>Research</b> variable",
)

fig.update_traces(diagonal_visible=False)

fig.show()

We can how students who have research experience tend to have higher scores and better GPA.

### Bivariate distribution with `University Rating`

As we saw earlier, the ranking of student's university plays an important role in other variables, and higher university ranking is linked with better performance in TOEFL and GRE exams, and higher GPA.

Here, we'll show the bivariate distribution of each pair of variables, for different university ranking values.

This way, we will be able to study the relation between two variables `x` and `y` as a function of the university ranking value.

#### `TOEFL Score` vs. `GRE Score`

In [ ]:
fig = px.density_heatmap(
    data_frame=df,
    x="TOEFL Score",
    y="GRE Score",
    color_continuous_scale=px.colors.sequential.Blugrn,
    facet_col="University Rating",
    title="<b>TOEFL Score</b> vs. <b>GRE Score</b> for different values of <b>University Rating</b>",
)

fig.show()

#### `TOEFL Score` vs. `CGPA`

In [ ]:
fig = px.density_heatmap(
    data_frame=df,
    x="TOEFL Score",
    y="CGPA",
    facet_col="University Rating",
    color_continuous_scale=px.colors.sequential.Blugrn,
    title="<b>TOEFL Score</b> vs. <b>CGPA</b> for different values of <b>University Rating</b>",
)

fig.show()

#### `GRE Score` vs. `CGPA`

In [ ]:
fig = px.density_heatmap(
    data_frame=df,
    x="GRE Score",
    y="CGPA",
    facet_col="University Rating",
    color_continuous_scale=px.colors.sequential.Blugrn,
    title="<b>GRE Score</b> vs. <b>CGPA</b> for different values of <b>University Rating</b>",
)

fig.show()

All these charts support our previous hypothesis: **University ranking influences positively student's performance.**

### `Research` and `University Rating`

In all previous plots we used either the `Research` variable or `University Rating` variable, however, it would very helpful to see how these two variables interact with each other, and how other vriables interact with them.

Here, we'll use scatterplot to show relation between two variables `x` and `y`, and show how this relation changes for different values of `Research` and `University Rating`.

This would be useful for answering questions:

- Do students who go to higher-ranking universitries, are more involved in research activiteis?
- How research experience and university ranking influence student's performance in exams and GPA?
- Are there any outlier points? 

#### `TOEFL Score` vs. `GRE Score`

In [ ]:
fig = px.scatter(
    data_frame=df,
    x="TOEFL Score",
    y="GRE Score",
    color="Research",
    facet_col="University Rating",
    trendline="ols",
    symbol="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    title="Scatter matrix for <b>TOEFL Score</b> vs. <b>GRE Score</b> conditioning on <b>University Rating</b> and <b>Research</b> variables",
)

fig.show()

#### `TOEFL Score` vs. `CGPA`

In [ ]:
fig = px.scatter(
    data_frame=df,
    x="TOEFL Score",
    y="CGPA",
    color="Research",
    facet_col="University Rating",
    trendline="ols",
    symbol="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    title="Scatter matrix for <b>TOEFL Score</b> vs. <b>CGPA</b> conditioning on <b>University Rating</b> and <b>Research</b> variables",
)

fig.show()

#### `GRE Score` vs. `CGPA`

In [ ]:
fig = px.scatter(
    data_frame=df,
    x="GRE Score",
    y="CGPA",
    color="Research",
    facet_col="University Rating",
    trendline="ols",
    symbol="Research",
    color_discrete_map={True: green_color_hex, False: red_color_hex},
    title="Scatter matrix for <b>GRE Score</b> vs. <b>CGPA</b> conditioning on <b>University Rating</b> and <b>Research</b> variables",
    labels={"University Rating": "University Rating"},
)

fig.show()

As we can see, higher university ranking is linked with research experience, and they both affects student's scores and GPA.

We can also see that there are some *outlier* points, where student goes to a low-ranking university and has no research experience, but have *good* scores.

This way of studying how the relation between two variables changes as a function of other variables is very useful when we have a very large amount of data, and we want to study the relation between two *quanitative variables*, and how this relation changes with respect to other *categorical variables*, as we will have fewer points to investigate, rather than a single scatter plot with too many points.